In [65]:
import os
import zipfile
import pandas as pd
import re

In [66]:
# Get the parent directory (one node above)
directory = os.path.dirname(os.getcwd())
print("Parent Directory:", directory)
os.listdir(directory)
# directory for original_loan_contracts
directory_loan = directory + '/Original_loan_contracts'
print("Loan Directory:", directory_loan)

Parent Directory: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements
Loan Directory: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts


In [67]:
# directory = '/Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/'

results = []

def search_after_phrase(content, phrase, offset, statement_keyword, fiscal_month_keyword, fiscal_year_keyword, projected_keyword, borrower_keyword):
    content = re.sub(r'\s+', ' ', content).strip()
    
    occurrences = [i for i in range(len(content)) if content.startswith(phrase, i)]
    monthly_fs_sentences = []
    projected_fs_sentences = []

    for start_index in occurrences:
        start_index += len(phrase)
        end_index = start_index + offset
        snippet = content[start_index:end_index]

        sentences = re.split(r'(?<=[.;]) +', snippet)

        for sentence in sentences:
            if (any(keyword in sentence for keyword in borrower_keyword) and any(keyword in sentence for keyword in fiscal_month_keyword) and any(keyword in sentence for keyword in statement_keyword)):
                monthly_fs_sentences.append(sentence)

            if (any(keyword in sentence for keyword in projected_keyword) and any(keyword in sentence for keyword in fiscal_year_keyword) and any(keyword in sentence for keyword in borrower_keyword)):
                projected_fs_sentences.append(sentence)

    return monthly_fs_sentences, projected_fs_sentences

import re

def search_after_phrase_meeting_inspection(content, phrase, offset, meeting_keyword, inspection_keyword):
    # Normalize whitespace in the content
    content = re.sub(r'\s+', ' ', content).strip()
    
    # Find all occurrences of the phrase in the content
    occurrences = [i for i in range(len(content)) if content.startswith(phrase, i)]
    
    # Initialize lists to store the extracted sentences
    lender_meeting_sentences = []
    inspection_sentences = []

    # Loop through each occurrence of the phrase
    for start_index in occurrences:
        # Move the start index to just after the phrase
        start_index += len(phrase)
        
        # Define the range to extract from (start_index to start_index + offset)
        end_index = start_index + offset
        snippet = content[start_index:end_index]

        # Split the snippet into sentences
        sentences = re.split(r'(?<=[.;]) +', snippet)
        
        # Search each sentence for keywords and extract only sentence
        for sentence in sentences:
            if (any(keyword in sentence for keyword in meeting_keyword)):
                lender_meeting_sentences.append(sentence)

            if (any(keyword in sentence for keyword in inspection_keyword)):
                inspection_sentences.append(sentence)

    return lender_meeting_sentences, inspection_sentences

# Set path to the 'Outputs/Agreements.csv' file in the parent directory
file_path = os.path.join(directory, 'Outputs', 'Agreements.csv')
result_df = pd.read_csv(file_path)
file_list = result_df['File Name'].tolist()

for filename in os.listdir(directory_loan):
    if filename.endswith('.zip'):
        zip_filepath = os.path.join(directory_loan, filename)
        print("Reading file:", zip_filepath)
        
        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            for file_to_read in file_list:
                if file_to_read in zip_ref.namelist():
                    with zip_ref.open(file_to_read) as file:
                        file_content = None
                        for encoding in ['utf-8', 'latin-1', 'utf-16']:
                            try:
                                file_content = file.read().decode(encoding)
                                break
                            except UnicodeDecodeError:
                                continue
                                
                        if file_content is None:
                            continue
                        
                        statement_keyword = [
                            "balance sheet", "income statement", "cash flow", "certificate", 
                            "statement of operations", "financial statement", "financial report"
                        ]

                        fiscal_month_keyword = [
                             "fiscal month", "monthly", "calendar month"
                        ]


                        fiscal_year_keyword = [
                            "fiscal year", "annually", "annual", "yearly", "calendar year"
                        ]
                        
                        projected_keyword = [
                            "forecast", "operating plan", "business plan", "projection", "budget", "projected", "budgeted"
                        ]
                        
                        borrower_keyword = [
                            "borrower", "borrowing", "agent", "lender", "parent", "holdings", "guarantor", "collateral property"
                        ]

                        meeting_keyword = [
                            "lender meeting", "lender call", "conference call", "conference meeting"
                        ]

                        inspection_keyword = [
                            "books and records", "inspection rights", "visitation rights", "examination rights", "rights to inspect", "inspection of books"
                        ]
                        
                        monthly_fs_sentences, projected_fs_sentences = search_after_phrase(
                            file_content, 'covenants', 10000, statement_keyword, fiscal_month_keyword, fiscal_year_keyword, projected_keyword, borrower_keyword
                        )

                        lender_meeting_sentences, inspection_sentences = search_after_phrase_meeting_inspection(
                            file_content, 'covenants', 20000, meeting_keyword, inspection_keyword
                        )

                        for sentence in monthly_fs_sentences:
                            results.append({
                                'filename': file_to_read,
                                'monthly_fs': 1,
                                'monthly_context': sentence,
                                'projected_fs': 0,
                                'projected_context': '',
                                'lender_meeting': 0,
                                'lender_meeting_context': ''
                            })

                        for sentence in projected_fs_sentences:
                            results.append({
                                'filename': file_to_read,
                                'monthly_fs': 0,
                                'monthly_context': '',
                                'projected_fs': 1,
                                'projected_context': sentence,
                                'lender_meeting': 0,
                                'lender_meeting_context': ''
                            })

                        for sentence in lender_meeting_sentences:
                            results.append({
                                'filename': file_to_read,
                                'monthly_fs': 0,
                                'monthly_context': '',
                                'projected_fs': 0,
                                'projected_context': '',
                                'lender_meeting': 1,
                                'lender_meeting_context': sentence
                            })

df = pd.DataFrame(results)
df = df.drop_duplicates()
# save the results to a csv file
df.to_csv(os.path.join(directory, 'Outputs', 'test.csv'), index=False)

Reading file: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts/2011-2015.zip
Reading file: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts/2020-2023.zip
Reading file: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts/2001-2010.zip
Reading file: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts/2016-2019.zip
Reading file: /Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts/1995-2000.zip


In [68]:
# summary of result_df
print(result_df.head())
print(result_df.shape)
print(result_df.columns)


                                         File Name       Search Term
0  2011/2011_1/20110329.nc/0001193125-11-081071.nc  credit agreement
1  2011/2011_1/20110329.nc/0000922224-11-000035.nc  credit agreement
2  2011/2011_1/20110329.nc/0001193125-11-081452.nc    loan agreement
3  2011/2011_1/20110329.nc/0001104659-11-017465.nc  credit agreement
4  2011/2011_1/20110329.nc/0001193125-11-081504.nc  credit agreement
(15335, 2)
Index(['File Name', 'Search Term'], dtype='object')
